# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,251.99,97,99,2.57,RU,1736375547
1,1,edinburgh of the seven seas,-37.0676,-12.3116,290.71,98,88,4.66,SH,1736375549
2,2,saipan,15.1355,145.7010,300.54,85,75,5.66,MP,1736375550
3,3,pathein,16.7833,94.7333,291.71,86,4,2.56,MM,1736375551
4,4,dzhalagash,45.0833,64.6667,265.34,69,3,3.68,KZ,1736375552


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    color='Humidity', 
    cmap='cool', 
    alpha=0.6, 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600, 
    hover_cols=['City', 'Humidity'])


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= 200) & 
    (city_data_df["Max Temp"] <= 300) & 
    (city_data_df["Wind Speed"] < 1) & 
    (city_data_df["Cloudiness"] < 17)
]


# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
71,71,newman,37.3138,-121.0208,289.28,43,12,0.89,US,1736375633
72,72,laguna,38.4210,-121.4238,290.43,50,0,0.89,US,1736375416
88,88,bilibino,68.0546,166.4372,238.24,99,11,0.50,RU,1736375655
229,229,eureka,40.8021,-124.1637,288.82,78,14,0.45,US,1736375825
254,254,ghadamis,30.1337,9.5007,282.39,55,0,0.18,LY,1736375854


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
71,newman,US,37.3138,-121.0208,43,
72,laguna,US,38.4210,-121.4238,50,
88,bilibino,RU,68.0546,166.4372,99,
229,eureka,US,40.8021,-124.1637,78,
254,ghadamis,LY,30.1337,9.5007,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,  
    "limit": 1

    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
newman - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
bilibino - nearest hotel: No hotel found
eureka - nearest hotel: Eureka Inn, Trademark Collection by Wyndham
ghadamis - nearest hotel: فندق عين الفارس
cill airne - nearest hotel: The Parkavon Hotel
barakani - nearest hotel: Al Amal
ardmore - nearest hotel: Best Western Plus
kharan - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
moranbah - nearest hotel: No hotel found
chittaurgarh - nearest hotel: Hotel Laxmi Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
71,newman,US,37.3138,-121.0208,43,No hotel found
72,laguna,US,38.4210,-121.4238,50,Holiday Inn Express & Suites
88,bilibino,RU,68.0546,166.4372,99,No hotel found
229,eureka,US,40.8021,-124.1637,78,"Eureka Inn, Trademark Collection by Wyndham"
254,ghadamis,LY,30.1337,9.5007,55,فندق عين الفارس


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size='Humidity', 
    color='Humidity', 
    cmap='cool', 
    alpha=0.6, 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600, 
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)